In [ ]:
## Carga de las librerías y clases que vamos a utilizar
import numpy as np
import pandas as pd
from datetime import datetime
import joblib
from sklearn.preprocessing import StandardScaler

In [ ]:
def eval_model(model=None, StartDate='2024-09-21 00:00:00', EndDate='2024-01-01 00:00:00'):

    # Autenticación del usuario para uso de la librería cryptoengineersdk
    user_token = {
        "user": "hector",
        "token": "a9agHyfg5478GfufUfj98534fs4gHh89Ig7v6fG89kJy7U5f5FFhjU88"
    }
    
    StartDate = datetime.strptime(StartDate, '%Y-%m-%d %H:%M:%S') 
    EndDate = datetime.strptime(EndDate, '%Y-%m-%d %H:%M:%S')
    
    # Parámetros para la obtención de la moneda cuyos precios han de predecirse
    table_name = 'silver_t_cryptos'
    symbol = 'XBTUSD'
    FI = StartDate.year
    FF = EndDate.year
    # Carga de la información sobre la criptomoneda que ha de predecirse
    df_crypto = ce.reader(user_token, table_name, symbol, FI, FF)

    table_name = 'silver_t_indices'
    symbol = 'SP500'
    FI = StartDate.year
    FF = EndDate.year
    # Carga de la información sobre la criptomoneda que ha de predecirse
    df_SP500 = ce.reader(user_token, table_name, symbol, FI, FF)
    
    table_name = 'silver_t_indices'
    symbol = 'Nikkei225'
    FI = StartDate.year
    FF = EndDate.year
    # Carga de la información sobre la criptomoneda que ha de predecirse
    df_Nikkei = ce.reader(user_token, table_name, symbol, FI, FF)

    table_name = 'silver_t_indices'
    symbol = 'IBEX35'
    FI = StartDate.year
    FF = EndDate.year
    # Carga de la información sobre la criptomoneda que ha de predecirse
    df_IBEX35 = ce.reader(user_token, table_name, symbol, FI, FF)

    table_name = 'silver_t_indices'
    symbol = 'Nasdaq'
    FI = StartDate.year
    FF = EndDate.year
    # Carga de la información sobre la criptomoneda que ha de predecirse
    df_Nasdaq = ce.reader(user_token, table_name, symbol, FI, FF)
    
    table_name = 'silver_t_indices'
    symbol = 'DAX'
    FI = StartDate.year
    FF = EndDate.year
    # Carga de la información sobre la criptomoneda que ha de predecirse
    df_DAX = ce.reader(user_token, table_name, symbol, FI, FF)

    # Definición de las variables para recoger los activos e indicadores cuyas cotizaciones se rescatarán de la base de datos  
    crypto_list = ["XBTUSD", "ETHUSD", "LINKUSD", "XDGUSD"]

    def load_crypto(df, crypto_file):
        
        # Definición del título de las columnas
        columns_names = [
            "Base_currency",
            "Type",
            "DateTime",
            "Date",
            "Time",
            "Frequency",
            "Month",
            "Day",
            f"{crypto_file}_Open",
            f"{crypto_file}_High",
            f"{crypto_file}_Low",
            f"{crypto_file}_Close", 
            f"{crypto_file}_Volume_USD", 
            f"{crypto_file}_Trades",
            "Audit_Time",
            "Symbol",
            "Year",
        ]
      
        # Añadir el nombre de las columnas al dataset
        df.columns = columns_names
        
        # Seleccionar únicamente las columnas con las que vamos a trabajar: una columna de fecha y otra columna con el precio de cierre para cada período temporal.
        df = df[["DateTime", f"{crypto_file}_Close"]]
        
        # Eliminar primera fila, correspondiente al encabezado existente en la tabla por defecto, que contiene nombres de columna inadecuados para nuestra tarea.
        df = df.drop(index=0).reset_index(drop=True)
        
        # Convertir en formato fecha la columna "DateTime".
        df["DateTime"] = pd.to_datetime(df["DateTime"])
        
        # Ordenar los registros en función del DateTime, esto es, cronológicamente
        df = df.sort_values(by= "DateTime")
        
        # Convertir columna de fecha en índice
        df = df.set_index("DateTime")
        
        # Castear los valores de las cotizaciones a tipo "float"
        for column in df.columns:
            df[column] = df[column].astype("float64")
    
        # Eliminar filas con índices duplicados, manteniendo solo la primera entrada/cotización
        df = df[~df.index.duplicated(keep="first")]
        
        return df
      
    # Carga del fichero de la criptomoneda cuyo valor ha de precedirse
    crypto_file = crypto_list[0]
    df_crypto = load_crypto(df_crypto, crypto_file)
 
    # Eliminar filas con índices duplicados, manteniendo solo la primera entrada/cotización
    df_crypto = df_crypto[~df_crypto.index.duplicated(keep="first")]                          

    # Modificación del título de las columnas para el índice S&P 500
    SP500_columns_names = ["Base_currency", "Type","DateTime", "Date", "Time", "Frequency" ,"Month", "Day", "SP500_Open",  "SP500_High",
                            "SP500_Low", "SP500_Close", "SP500_Volume_USD", "SP500_Trades", "Audit_Time", "Symbol", "Year"]
    df_SP500.columns = SP500_columns_names
    # Modificación del título de las columnas para el índice Nikkei-225
    Nikkei_columns_names = ["Base_currency", "Type","DateTime", "Date", "Time", "Frequency" ,"Month", "Day", "Nikkei_Open",  "Nikkei_High",
                            "Nikkei_Low", "Nikkei_Close", "Nikkei_Volume_USD", "Nikkei_Trades", "Audit_Time", "Symbol", "Year"]
    df_Nikkei.columns = Nikkei_columns_names
    # Modificación del título de las columnas para el índice IBEX-35
    IBEX35_columns_names = ["Base_currency", "Type","DateTime", "Date", "Time", "Frequency" ,"Month", "Day", "IBEX35_Open",  "IBEX35_High",
                            "IBEX35_Low", "IBEX35_Close", "IBEX35_Volume_USD", "IBEX35_Trades", "Audit_Time", "Symbol", "Year"]
    df_IBEX35.columns = IBEX35_columns_names
    # Modificación del título de las columnas para el índice NASDAQ-100
    Nasdaq_columns_names = ["Base_currency", "Type","DateTime", "Date", "Time", "Frequency" ,"Month", "Day", "Nasdaq_Open",  "Nasdaq_High",
                            "Nasdaq_Low", "Nasdaq_Close", "Nasdaq_Volume_USD", "Nasdaq_Trades", "Audit_Time", "Symbol", "Year"]
    df_Nasdaq.columns = Nasdaq_columns_names    
    # Modificación del título de las columnas para el índice DAX (Alemania)
    DAX_columns_names = ["Base_currency", "Type","DateTime", "Date", "Time", "Frequency" ,"Month", "Day", "DAX_Open",  "DAX_High",
                            "DAX_Low", "DAX_Close", "DAX_Volume_USD", "DAX_Trades", "Audit_Time", "Symbol", "Year"]
    df_DAX.columns = DAX_columns_names

    # Seleccionar únicamente las columnas que necesitamos
    df_SP500 = df_SP500.iloc[:, [3,4,11]] 
    df_Nikkei = df_Nikkei.iloc[:, [3,4,11]]
    df_IBEX35 = df_IBEX35.iloc[:, [3,4,11]]
    df_Nasdaq = df_Nasdaq.iloc[:, [3,4,11]]
    df_DAX = df_DAX.iloc[:, [3,4,11]]
  
    # Eliminar primera fila, correspondiente al encabezado por defecto.
    df_SP500 = df_SP500.drop(index=0).reset_index(drop=True)
    df_Nikkei = df_Nikkei.drop(index=0).reset_index(drop=True)
    df_IBEX35 = df_IBEX35.drop(index=0).reset_index(drop=True)
    df_Nasdaq = df_Nasdaq.drop(index=0).reset_index(drop=True)
    df_DAX = df_DAX.drop(index=0).reset_index(drop=True)    

    # Concatenar las columnas "Date" y "Time", para hacer cuadrar la estructura de los datos de fecha con la del resto de datasets
    df_SP500["DateTime"] = pd.to_datetime(df_SP500["Date"] + ' ' + df_SP500["Time"])    
    df_Nikkei["DateTime"] = pd.to_datetime(df_Nikkei["Date"] + ' ' + df_Nikkei["Time"])
    df_IBEX35["DateTime"] = pd.to_datetime(df_IBEX35["Date"] + ' ' + df_IBEX35["Time"])
    df_Nasdaq["DateTime"] = pd.to_datetime(df_Nasdaq["Date"] + ' ' + df_Nasdaq["Time"])
    df_DAX["DateTime"] = pd.to_datetime(df_DAX["Date"] + ' ' + df_DAX["Time"])
 
    # Eliminar las columnas Date y Time, que ya no vamos a utilizar
    df_SP500 = df_SP500[["DateTime", "SP500_Close"]]
    df_Nikkei = df_Nikkei[["DateTime", "Nikkei_Close"]]
    df_IBEX35 = df_IBEX35[["DateTime", "IBEX35_Close"]]
    df_Nasdaq = df_Nasdaq[["DateTime", "Nasdaq_Close"]]
    df_DAX = df_DAX[["DateTime", "DAX_Close"]]
  
    # Convertir columnas de fecha en índices
    df_SP500 = df_SP500.set_index("DateTime")
    df_Nikkei = df_Nikkei.set_index("DateTime")
    df_IBEX35 = df_IBEX35.set_index("DateTime")
    df_Nasdaq = df_Nasdaq.set_index("DateTime")
    df_DAX = df_DAX.set_index("DateTime")

    # Castear valores a flotantes
    df_SP500.iloc[:, 0] = df_SP500.iloc[:, 0].astype("float64")
    df_Nikkei.iloc[:, 0] = df_Nikkei.iloc[:, 0].astype("float64")
    df_IBEX35.iloc[:, 0] = df_IBEX35.iloc[:, 0].astype("float64")
    df_Nasdaq.iloc[:, 0] = df_Nasdaq.iloc[:, 0].astype("float64")
    df_DAX.iloc[:, 0] = df_DAX.iloc[:, 0].astype("float64")
 
    # Desindexar la columna DateTime para limpiarla y darle formato de fecha
    df_SP500 = df_SP500.reset_index()
    df_Nikkei = df_Nikkei.reset_index()
    df_IBEX35 = df_IBEX35.reset_index()
    df_Nasdaq = df_Nasdaq.reset_index()
    df_DAX = df_DAX.reset_index()

    # Dar formato de fecha a la columna DateTime
    df_SP500["DateTime"] = pd.to_datetime(df_SP500["DateTime"], format = "%Y-%m-%d %H:%M:%S")    
    df_Nikkei["DateTime"] = pd.to_datetime(df_Nikkei["DateTime"], format = "%Y-%m-%d %H:%M:%S")
    df_IBEX35["DateTime"] = pd.to_datetime(df_IBEX35["DateTime"], format = "%Y-%m-%d %H:%M:%S")
    df_Nasdaq["DateTime"] = pd.to_datetime(df_Nasdaq["DateTime"], format = "%Y-%m-%d %H:%M:%S")
    df_DAX["DateTime"] = pd.to_datetime(df_DAX["DateTime"], format = "%Y-%m-%d %H:%M:%S")
   
    # Reindexar la columna DateTime
    df_SP500 = df_SP500.set_index("DateTime")    
    df_Nikkei = df_Nikkei.set_index("DateTime")
    df_IBEX35 = df_IBEX35.set_index("DateTime")
    df_Nasdaq = df_Nasdaq.set_index("DateTime")
    df_DAX = df_DAX.set_index("DateTime")

    # Fusión de los datasets de las variables predictoras en función de la columna "DateTime", incluyendo temnporalmente el dataset del Bitcoin, para poder hacer ffill y bfill del Nikkei basado en el índice de fechas de la criptomoneda.
    df_predictors = (
        df_crypto
        .merge(df_SP500, how="left", on="DateTime")
        .merge(df_Nikkei, how="left", on="DateTime")
        .merge(df_IBEX35, how="left", on="DateTime")
        .merge(df_Nasdaq, how="left", on="DateTime")        
        .merge(df_DAX, how="left", on="DateTime")
    )

    # Bucle para rellenar los valores faltantes en las variables, replicando el primer valor posterior disponible si hay valores faltantes al inicio del dataframe o el valor anterior disponible. 
    for col in df_predictors.columns:
        df_predictors[col] = df_predictors[col].fillna(method = "bfill").fillna(method = "ffill")

    # Eliminamos del dataframe la columna de Bitcoin, ya que es la variable que se debe predecir y no una predictora.
    df_predictors = df_predictors.drop(columns="XBTUSD_Close")

    # Definición del escalador
    sc_X = StandardScaler()
    # Escalado de los valores de las variables predictoras
    nn_predictors_sc = sc_X.fit_transform(df_predictors)

    # Predicciones con el modelo de regresión lineal múltiple cargado
    nn_predictions = model.predict(nn_predictors_sc)

    # Convertir la serie a Dataframe y renombrar la columna.
    df_px_pred = pd.DataFrame(nn_predictions)
    df_px_pred = df_px_pred.rename(columns = {0:"XBTUSD_Close_prediction"})   


    """ESTRATEGIA DE TRADING"""

    # Convertir las columnas a tipo numérico
    df_px_pred["XBTUSD_Close_prediction"] = pd.to_numeric(df_px_pred["XBTUSD_Close_prediction"], errors='coerce')
    df_crypto["XBTUSD_Close"] = pd.to_numeric(df_crypto["XBTUSD_Close"], errors='coerce')

    # Creación de variables necesarias para la operativa.
    open_position = False
    purchase_price = 0
    last_signal = 0
    
    # Lista para almacenar las señales
    signals = []
    
    # Definir los porcentajes de stop loss y take profit
    stop_loss_pct = 0.05  # Venta automática si se genera un 5% de pérdida
    take_profit_pct = 0.10  # Venta automática si se genera un 10% de beneficio
    
    for i, prediction in enumerate(df_px_pred["XBTUSD_Close_prediction"]):
        # Verificar si el dataframe que contiene las cotizaciones reales tiene suficientes filas
        if i >= len(df_crypto):
            # Terminar el bucle si el dataframe es más corto.
            break  
    
        # Definir variable para el precio predicho en el intervalo anterior
        current_price = df_px_pred["XBTUSD_Close_prediction"].iloc[i-1]
    
        # Asegurarse de que no hay valores faltantes antes de hacer la comparación
        if pd.notna(prediction) and pd.notna(current_price):
            # Condición para comprar
            if not open_position and prediction > current_price:
                # Señal de compra
                signals.append(1)
                purchase_price = current_price
                open_position = True
            # Condición para vender (stop loss o take profit)
            elif open_position:
                # Stop loss: si el precio cae más de un 5% desde la compra
                if current_price <= purchase_price * (1 - stop_loss_pct):
                    # Señal de venta
                    signals.append(0)
                    open_position = False
                # Take profit: si el precio sube más de un 10% desde la compra
                elif current_price >= purchase_price * (1 + take_profit_pct):
                    # Señal de venta
                    signals.append(0)
                    open_position = False
                else:
                    # Mantener la posición de compra si no se cumplen las condiciones de venta
                    signals.append(1)
            else:
                # No hay posición abierta y no hay señal de compra
                signals.append(0)
        else:
            # Manejar los casos en los que no hay acción de compraventa
            signals.append(0)
    
    # Ajustar señales a la longitud del dataframe de cotizaciones reales de la criptomoneda, en caso de desajuste.
    if len(signals) < len(df_crypto):
        # Rellenar con 0 (sin acción) si faltan señales
        signals.extend([0] * (len(df_crypto) - len(signals)))
    
    # Crear dataframe de salida
    df_outputs = pd.DataFrame({
        "DATETIME": pd.to_datetime(df_crypto.index),
        "CLOSE": df_crypto["XBTUSD_Close"],
        "SIGNAL": signals[:len(df_crypto)]
    })
    
    # Eliminar el índice (fechas), por requerimiento de la función de evaluación del modelo (eval_model)
    df_outputs = df_outputs.reset_index(drop=True)

    # Mostrar el dataframe de salida
    return df_outputs